# 要約 
このJupyter Notebookでは、Kaggleの「20の質問」ゲームにおけるエージェントの開発と実行を目的としています。特に、4つの異なるエージェントを定義し、それらが様々なカテゴリ（電子機器、食品、乗り物、国）に関連する質問をランダムに生成し、ターゲットとなる単語を推測するプロセスに取り組んでいます。

### 主な問題
ノートブックは「20の質問」ゲームを模倣したエージェントの設計に関連しており、エージェントがどのようにしてターゲット単語を特定するために質問を行うか、またその回答によって次の質問を決定するかを扱っています。

### 使用手法
- **エージェントの実装**: 各エージェントは、特定のカテゴリーに対する質問を持ち、その質問に基づいてターゲットを絞り込むロジックを持っています。エージェントは「質問をする」、「推測する」、「回答する」というターンに応じて異なる行動を取ります。
- **ヘルパー関数**: カテゴリのキーワードに対して関連付けを判断するための関数`is_related`が定義されています。これは、キーワードが特定のカテゴリーに関連しているかを評価するために使用されます。

### 使用ライブラリ
- **NumPy**: 数値計算に使用。
- **Pandas**: データ処理とCSVファイルの操作に使用。
- **Kaggle Environments**: ゲーム環境を作成し、エージェントを実行するために使用され、エージェントの推測や対戦の結果が収集されます。

### 実行と提出準備
ゲームの環境を作成し、定義されたエージェントを使用してゲームを実行し、その結果を可視化します。また、エージェントの実装コードを`main.py`ファイルに書き込み、提出用のアーカイブファイルを作成する準備を整えています。

全体として、このノートブックは20の質問ゲームにおけるAIエージェントの設計と実行の流れを示しており、特に戦略的な質問と答えの生成に焦点を当てています。

---


# 用語概説 
以下は、Jupyter Notebookからの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語の解説リストです。比較的一般的な内容は含まず、特に実務経験が少ない人が理解しにくい用語に焦点を当てています。

1. **エージェント (Agent)**:
   - ゲームやシミュレーション環境内で決定を下すプログラムやモデルを指します。この場合、エージェントは質問を行ったり、回答を提供するためのロジックを持った関数です。

2. **観測 (Observation)**:
   - エージェントが環境から受け取る情報のこと。ゲームの進行や状態に応じて、現在のラウンドの情報を含みます。

3. **ターン (Turn)**:
   - ゲーム内での行動の単位で、エージェントが質問をしたり、回答をしたりする各ステップを指します。ターンの種類に応じて、エージェントの動作が変わります。

4. **カテゴリキーワード (Category Keywords)**:
   - 特定のテーマに関連した単語のリスト。例えば、電子機器カテゴリのキーワードには「スマートフォン」や「ラップトップ」があります。

5. **環境 (Environment)**:
   - エージェントが相互作用する領域やシステム。その動作やルールを定義し、エージェントが行動するための条件を提供します。この場合、ゲーム「20の質問」のルールを含みます。

6. **デフォルト (Default)**:
   - 何かを設定する際の初期状態や選択肢を指します。例えば、`make()`関数のデフォルトオプションを使用することで、簡略化された設定が可能です。

7. **代替 (Alternates)**:
   - 特定のキーワードに対する代替の推測候補を示すもの。例えば、あるキーワードに対し、複数の可能性を提示することで、エージェントがより効果的に推測できるようにします。

8. **推測 (Guess)**:
   - エージェントがターゲットとなる単語を特定するための推定を行う行為。正しい単語を当てるために仮説を立て、最終的な回答を行います。

9. **ヘルパー関数 (Helper Function)**:
   - メインのロジックを支援するために作成された関数。特定のタスクを簡素化し、再利用可能なコードを提供します。

10. **ロジック (Logic)**:
    - エージェントが質問を組み立てたり、推測を行ったりするための一連の判断基準や規則のこと。状態に応じて異なるアクションを取るための条件分岐を含むことが多いです。

これらの解説は、特に初心者がつまずきやすい点をカバーしており、競技プログラミングや機械学習における用語の理解を助けるための参考になるでしょう。

---


In [ ]:
# このPython 3環境には、多くの便利な解析ライブラリがインストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここではいくつかの便利なパッケージを読み込みます

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは、読み取り専用の "../input/" ディレクトリにあります
# 例えば、これを実行すると (実行するにはクリックするか Shift+Enter を押します)、入力ディレクトリ内のすべてのファイルをリストします

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ (/kaggle/working/) に最大20GBまで書き込むことができ、"Save & Run All" を使用してバージョンを作成すると出力として保存されます 
# 一時ファイルは /kaggle/temp/ に書き込むこともできますが、そのセッション外では保存されません

In [ ]:
import random

# キーワードがカテゴリに関連しているかをチェックするヘルパー関数
def is_related(keyword, category_keywords):
    return keyword.lower() in category_keywords

# カテゴリキーワード
electronics_keywords = {"スマートフォン", "ラップトップ", "タブレット", "テレビ"}
food_keywords = {"ピザ", "バーガー", "寿司", "パスタ"}
vehicles_keywords = {"車", "バイク", "飛行機", "ボート"}
countries_keywords = {"日本", "フランス", "ブラジル", "カナダ"}

def simple_agent1(obs, cfg):
    electronics_questions = [
        "それはスマートフォンですか?", "それはラップトップですか?", 
        "それはタブレットですか?", "それはテレビですか?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(electronics_questions)
    elif obs.turnType == "guess": 
        response = "スマートフォン"  # ここに改善されたロジックを追加できます
    elif obs.turnType == "answer":
        response = "**はい**" if is_related(obs.keyword, electronics_keywords) else "**いいえ**"
    return response

def simple_agent2(obs, cfg):
    food_questions = [
        "それはピザですか?", "それはバーガーですか?", 
        "それは寿司ですか?", "それはパスタですか?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(food_questions)
    elif obs.turnType == "guess": 
        response = "ピザ"  # ここに改善されたロジックを追加できます
    elif obs.turnType == "answer":
        response = "**はい**" if is_related(obs.keyword, food_keywords) else "**いいえ**"
    return response

def simple_agent3(obs, cfg):
    vehicle_questions = [
        "それは車ですか?", "それはバイクですか?", 
        "それは飛行機ですか?", "それはボートですか?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(vehicle_questions)
    elif obs.turnType == "guess": 
        response = "車"  # ここに改善されたロジックを追加できます
    elif obs.turnType == "answer":
        response = "**はい**" if is_related(obs.keyword, vehicles_keywords) else "**いいえ**"
    return response

def simple_agent4(obs, cfg):
    country_questions = [
        "それは日本ですか?", "それはフランスですか?", 
        "それはブラジルですか?", "それはカナダですか?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(country_questions)
    elif obs.turnType == "guess": 
        response = "日本"  # ここに改善されたロジックを追加できます
    elif obs.turnType == "answer":
        response = "**はい**" if is_related(obs.keyword, countries_keywords) else "**いいえ**"
    return response

# ゲーム環境の作成と設定

Kaggleの環境は、`make()`関数を使用して作成されます。ここでは、*環境*名 (`"llm_20_questions"`) と、いくつかのオプションのデフォルト（*構成*や*情報*など）を指定します。コンペティションと同じようにゲームを実行したい場合は、デフォルトをそのまま使用できます。


In [ ]:
import kaggle_environments
env = kaggle_environments.make(environment="llm_20_questions")
# （「No pygame installed」というエラーは無視しても構いません）

In [ ]:
print("このセッションのキーワードは: ")
print(kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword)
print(" ")
print("いくつかのキーワードには、代替の推測（alts）のリストもあります。")
print("このセッションの代替リストは:")
print(kaggle_environments.envs.llm_20_questions.llm_20_questions.alts)

# LLM 20 Questions の実行（デフォルト）

作成した環境 (`env`) でゲームを実行します。ゲームを実行する際は、4つのエージェントのリストを提出する必要があります: 
* "エージェント1" （チーム1の推測者）、 
* "エージェント2" （チーム1の回答者）、 
* "エージェント3" （チーム2の推測者）、 
* "エージェント4" （チーム2の回答者）。 

コンペティションでは、ランダムに選ばれたチームメイトと対戦し、推測者または回答者のどちらかになります。


In [ ]:
%%time
game_output = env.run(agents=[simple_agent1, simple_agent2, simple_agent3, simple_agent4])

In [ ]:
env.render(mode="ipython", width=600, height=500)

# 提出可能なエージェントの作成
コンペティションにエージェントを提出するには、エージェントのPythonコードをmain.pyというファイルに書き、そのファイルとサポートファイルをsubmission.tar.gzにまとめる必要があります。

以下はシンプルな例です。もちろん、実際のコンペティションでは、公式のスターターノートブック（https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook）で示されているように、実際のLLMを使用することになるでしょう。ノートブックでLLMエージェントを実行するのは、時間とメモリがかかるため、テストとしてプレイヤー1にシンプルなエージェントを使用したい場面もあるでしょう。


In [ ]:
import os
submission_directory = "/kaggle/working/submission"
submission_subdirectory = "lib"
# メインディレクトリが存在しない場合は作成
if not os.path.exists(submission_directory):
    os.mkdir(submission_directory)
    subdirectory_path = os.path.join(submission_directory, submission_subdirectory)
    os.mkdir(subdirectory_path)

In [ ]:
import os
import csv

# メインディレクトリとサブディレクトリを定義
submission_directory = "/kaggle/working/submission"
submission_subdirectory = "lib"

# メインディレクトリが存在しない場合は作成
if not os.path.exists(submission_directory):
    os.mkdir(submission_directory)

# サブディレクトリの完全なパスを定義
subdirectory_path = os.path.join(submission_directory, submission_subdirectory)

# サブディレクトリが存在しない場合は作成
if not os.path.exists(subdirectory_path):
    os.mkdir(subdirectory_path)

# libディレクトリに例示用のCSVファイルを作成
with open(os.path.join(subdirectory_path, "example.csv"), mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(["牛", "馬"])

* エージェントのためのmain.py Pythonコードを書く
* 環境はmain.pyの最後の関数をエージェントとして使用します。この場合は`agent_fun()`です。



In [ ]:
%%writefile /kaggle/working/submission/main.py

import os
import sys
import csv
import random



# 提出/libディレクトリに他のファイル（例：モデルの重み）を置く場合は、パスを設定する必要があります
KAGGLE_COMPETITION_PATH = "/kaggle_simulations/agent/" # コンペティションパス
if os.path.exists(KAGGLE_COMPETITION_PATH):  # コンペティション内で実行中の場合
    subdirectory_path = os.path.join(KAGGLE_COMPETITION_PATH, "lib")
else: # ノートブックで実行中の場合
    subdirectory_path = os.path.join("/kaggle/working/submission/", "lib")
sys.path.insert(0, subdirectory_path)


# 例示用ファイルを読み込む
with open(os.path.join(subdirectory_path,"example.csv"), mode='r') as file:
    reader = csv.reader(file)
    guess_list = list(reader)
    guess_list = guess_list[0]

# 例示ファイルからランダムに選んだ"動物"をグローバル変数として設定
animal = random.choice(guess_list)
    
# main.py の最後の関数がエージェント関数となります
def agent_fn(obs, cfg):
    
    # エージェントが推測者で、turnType が "ask" の場合
    if obs.turnType == "ask":
        response = f'それは{animal}のように見えますか?'
    # エージェントが推測者で、turnType が "guess" の場合
    elif obs.turnType == "guess":
        if obs.answers[-1]=="yes":
            response = animal
        else:
            response = "ペンギン"
    # エージェントが回答者の場合
    elif obs.turnType == "answer":
        if obs.keyword in obs.questions[-1]:
            response = "はい"
        else:
            response = "いいえ"
        
    return response

この`main.py`ファイルとエージェントは、`/lib/example.csv`サポートファイルと共に提出する準備が整いました。


In [ ]:
!apt install pigz pv > /dev/null
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
game_output = env.run(agents=["/kaggle/working/submission/main.py", "/kaggle/working/submission/main.py", simple_agent3, simple_agent4])
env.render(mode="ipython", width=600, height=500)